# Spotify API Experiments

https://developer.spotify.com/community

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

# Spotify Credentials
import spot_creds

# Graphing
import matplotlib.pyplot as plt

# Scikit-Learn
from sklearn.feature_extraction.text import TfidfVectorizer

clid = spot_creds.client_id
secret = spot_creds.secret


In [2]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=clid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Get Songs from Playlist
Using my "Tom's Funky Jazz Group" playlist.

In [3]:
# playlist_link = "https://open.spotify.com/playlist/1jAT4AEQab8P4Qg1olncPw" # Tom's Funky Jazz Group
playlist_link = "https://open.spotify.com/playlist/7eWWLoTfmLUcD0viBP6Hr0?si=e8b0760749404749" #Funky Songs
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
# sp.playlist_tracks(playlist_URI)

In [5]:
first_track = sp.playlist_tracks(playlist_URI)["items"][0]

In [6]:
first_track

{'added_at': '2020-10-21T17:21:52Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/12100945354'},
  'href': 'https://api.spotify.com/v1/users/12100945354',
  'id': '12100945354',
  'type': 'user',
  'uri': 'spotify:user:12100945354'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2pXFmyqPm7wHJ1HGAwyR3L'},
     'href': 'https://api.spotify.com/v1/artists/2pXFmyqPm7wHJ1HGAwyR3L',
     'id': '2pXFmyqPm7wHJ1HGAwyR3L',
     'name': 'Here Come The Mummies',
     'type': 'artist',
     'uri': 'spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L'}],
   'available_markets': [],
   'external_urls': {'spotify': 'https://open.spotify.com/album/5qkp95o3LcCOpo4rQddh3n'},
   'href': 'https://api.spotify.com/v1/albums/5qkp95o3LcCOpo4rQddh3n',
   'id': '5qkp95o3LcCOpo4rQddh3n',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b2730c4072e9a5c

In [7]:
def extract_audio_feat(track_uri, track_dict):
        # Audio Features
    audio_feat_list = ['acousticness',
                       'danceability',
                       'energy',
                       'instrumentalness',
                       'key',
                       'liveness',
                       'loudness',
                       'mode',
                       'speechiness',
                       'tempo',
                       'time_signature',
                       'valence']
                       
    audio_feat = sp.audio_features(track_uri)[0]
    
    audio_feat_list = ['acousticness',
                       'danceability',
                       'energy',
                       'instrumentalness',
                       'key',
                       'liveness',
                       'loudness',
                       'mode',
                       'speechiness',
                       'tempo',
                       'time_signature',
                       'valence']
    
    for feat in audio_feat_list:
        track_dict[feat] = audio_feat[feat]
        
    return track_dict




In [8]:
tracks_df = pd.DataFrame()

for track in sp.playlist_tracks(playlist_URI)["items"]:
    this_track = {}
    #URI
    track_uri = track["track"]["uri"]
    this_track['track_uri'] = track_uri
    
    #Track name
    this_track['track_name'] = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    this_track['artist_uri'] = artist_uri
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    this_track['artist_name'] = track["track"]["artists"][0]["name"]
    this_track['artist_pop'] = artist_info["popularity"]
    this_track['artist_genres'] = artist_info["genres"]
    
    #Album
    this_track['album'] = track["track"]["album"]["name"]
    
    #Track Metadata
    this_track['track_pop'] = track["track"]["popularity"]
    this_track['explicit'] = track["track"]['explicit']
    
    # Audio Features
    this_track = extract_audio_feat(track_uri, this_track)
    
    
# Convert to DataFrame
    this_track_df = pd.json_normalize(this_track)
    
    tracks_df = pd.concat([tracks_df,this_track_df], ignore_index=True)
    
    # Make sure there are no duplicates
    tracks_df = tracks_df.drop_duplicates('track_uri')
    
tracks_df.head()

,track_uri,track_name,artist_uri,artist_name,artist_pop,artist_genres,album,track_pop,explicit,acousticness,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,spotify:track:5ZDLCdSHmpt2JKwWmotjzG,Pants,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,35,[funk rock],Single Entendre,0,False,0.02650,...,0.851,0.000000,1,0.3580,-4.067,0,0.0351,123.910,4,0.809
1,spotify:track:6NBqMK4DfBYGIzCnmSgliG,Dirty Minds,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,35,[funk rock],Everlasting Party,0,False,0.00999,...,0.832,0.000000,9,0.0902,-5.705,1,0.0605,132.901,4,0.878
2,spotify:track:4zxXklp6D4vClyC08SUeo9,Carnal Carnival,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,35,[funk rock],Carnal Carnival,0,False,0.44300,...,0.934,0.000000,11,0.6820,-3.049,0,0.0708,160.061,4,0.753
3,spotify:track:3qaO2Ko1b0bBisqNLU1e1s,Freak Flag,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,35,[funk rock],Carnal Carnival,0,False,0.03550,...,0.901,0.000000,1,0.3740,-3.393,1,0.0452,109.968,4,0.830
4,spotify:track:6zADCSrbXuHSTnZbznyyY9,Live in the Moment,spotify:artist:2pXFmyqPm7wHJ1HGAwyR3L,Here Come The Mummies,35,[funk rock],Live in the Moment,0,False,0.00324,...,0.753,0.000215,1,0.1740,-6.065,0,0.0329,105.947,4,0.502


In [ ]:
tracks_df.loc[tracks_df.explicit==True]

In [ ]:
genres_df = pd.DataFrame(
        pd.DataFrame(tracks_df.artist_genres.tolist())[0].tolist(), index=tracks_df.track_name
)
genres_df.columns=['genre']
genres_df.reset_index(inplace=True)
genres_df.head()

In [ ]:
genres_df.genre.value_counts().plot(kind='bar', rot=45)

In [ ]:
tracks_df.artist_name.value_counts()

## Audio Features

In [ ]:
tracks_df.iloc[0]

In [ ]:
track_uri = tracks_df.iloc[1]['track_uri']
audio_feat = sp.audio_features(track_uri)
audio_feat

In [ ]:
audio_feat[0]['danceability']

In [ ]:
# sp.audio_analysis(track_uri)

In [ ]:
tracks_df.describe()

## Features

### Meta Data

In [ ]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df



In [ ]:
# TF-IDF implementation
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(tracks_df['artist_genres'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
# genre_df.drop(columns='genre|unknown') # Drop unknown genre
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

In [ ]:
list(genre_df.columns)

In [ ]:
# normalize popularity
tracks_df['artist_pop'] = tracks_df['artist_pop']/100
tracks_df['track_pop'] = tracks_df['track_pop']/100


# Artist
Can we get "Fans Also Like" artists from an artist and then pull those songs?

In [20]:
artist_id = '2pXFmyqPm7wHJ1HGAwyR3L' # HCTM
artist_id = '7JnJgTo8cCtAQmtC0cJyjp' # Tom McGuire and the Brassholes
related_artists = sp.artist_related_artists(artist_id)

In [21]:
for artist in related_artists['artists']:
    print(artist['name'])

Family Company
Verdine White
Love the System
Philip Lassiter
Dumpstaphunk
Cissy Street
The Filthy Six
Funky Times
Redtenbacher's Funkestra
Supermoon
Orgone
Swatkins
Echo Echo
The Haggis Horns
The New Mastersounds
Sam Greenfield
Mingo Fishtrap
Giacomo Turra
Dylan Chambers
Huntertones
